<a href="https://colab.research.google.com/github/Cinichi/Ani-Downloader/blob/main/Manga_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# 🎬 AnimeKai Episode Downloader & Merger
# Enhanced with chunk downloads, yt-dlp support, and better error handling

# @title 🔧 **Install Dependencies** { display-mode: "form" }
print("📦 Installing required packages...")
!pip install -q requests beautifulsoup4 cloudscraper m3u8 pycryptodome tqdm yt-dlp
!apt-get -qq install -y ffmpeg aria2 > /dev/null 2>&1
print("✅ All dependencies installed!\n")

# @title ⚙️ **Configuration** { display-mode: "form" }

#@markdown ### 🔗 Anime URL
#@markdown Enter the AnimeKai watch URL:
anime_url = "https://anikai.to/watch/attack-on-titan-season-3-oxgk" #@param {type:"string"}

#@markdown ---
#@markdown ### 📺 Episode Selection
download_mode = "All Episodes" #@param ["All Episodes", "Episode Range", "Single Episode"]

#@markdown Single episode number:
single_episode = 1 #@param {type:"integer"}

#@markdown Episode range (Start and End):
start_episode = 1 #@param {type:"integer"}
end_episode = 3 #@param {type:"integer"}

#@markdown ---
#@markdown ### 🎥 Quality & Audio Settings
video_quality = "1080p" #@param ["1080p", "720p", "480p", "360p"]
prefer_type = "Dub & S-Sub" #@param ["Hard Sub", "Soft Sub", "Dub & S-Sub"]
prefer_server = "Server 1" #@param ["Server 1", "Server 2"]

#@markdown ---
#@markdown ### 📥 Download Settings
download_method = "yt-dlp" #@param ["yt-dlp", "aria2", "chunks", "ffmpeg"]

#@markdown Chunk size in MB (for chunked downloads):
chunk_size_mb = 14 #@param {type:"slider", min:1, max:50, step:1}

#@markdown Max parallel workers/connections:
max_workers = 15 #@param {type:"slider", min:1, max:32, step:1}

#@markdown Max retry attempts:
max_retries = 5 #@param {type:"slider", min:1, max:10, step:1}

#@markdown Timeout in seconds:
timeout = 300 #@param {type:"slider", min:60, max:600, step:30}

#@markdown ---
#@markdown ### 🔗 Merge Settings
merge_episodes = True #@param {type:"boolean"}

#@markdown ---
#@markdown ### 📤 Upload Settings
upload_to = "GoFile.io Only" #@param ["Google Drive Only", "GoFile.io Only", "Both", "None (Keep Local)"]

print("✅ Configuration loaded!")

# @title 🌐 **Core Functions** { display-mode: "form" }

import requests
import re
import json
import os
import time
import subprocess
import threading
from bs4 import BeautifulSoup
import cloudscraper
from urllib.parse import urljoin, urlparse, quote, unquote
import shutil
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Create cloudscraper session
scraper = cloudscraper.create_scraper(
    browser={'browser': 'chrome', 'platform': 'windows', 'desktop': True}
)

BASE_URL = "https://animekai.to"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
    'Referer': BASE_URL,
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive'
}

def enc_dec_request(endpoint, text):
    """Make request to enc-dec API"""
    try:
        url = f"https://enc-dec.app/api/{endpoint}?text={text}"
        response = scraper.get(url, headers=headers, timeout=30)
        data = response.json()
        return data.get('result', '')
    except Exception as e:
        print(f"⚠️ Enc-dec error: {e}")
        return None

def get_anime_details(url):
    """Get anime ID and title"""
    try:
        response = scraper.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        anime_div = soup.select_one('div[data-id]')
        anime_id = anime_div.get('data-id') if anime_div else None

        title_elem = soup.select_one('div.title-wrapper h1.title span')
        title = title_elem.get('title', '') if title_elem else "Unknown"
        title = re.sub(r'[<>:"/\\|?*]', '', title)

        return anime_id, title
    except Exception as e:
        print(f"❌ Error getting anime details: {e}")
        return None, None

def get_episode_list(anime_id):
    """Get list of all episodes"""
    try:
        enc = enc_dec_request('enc-kai', anime_id)
        if not enc:
            return []

        ep_url = f"{BASE_URL}/ajax/episodes/list?ani_id={anime_id}&_={enc}"
        response = scraper.get(ep_url, headers=headers)
        data = response.json()

        if 'result' not in data:
            return []

        html = data['result']
        soup = BeautifulSoup(html, 'html.parser')

        episodes = []
        for ep in soup.select('div.eplist a'):
            token = ep.get('token', '')
            ep_num = ep.get('num', '0')
            langs = ep.get('langs', '0')

            langs_int = int(langs) if langs.isdigit() else 0
            if langs_int == 1:
                subdub = "Sub"
            elif langs_int == 3:
                subdub = "Dub & Sub"
            else:
                subdub = ""

            episodes.append({
                'number': float(ep_num),
                'token': token,
                'subdub': subdub,
                'title': f"Episode {ep_num}"
            })

        return sorted(episodes, key=lambda x: x['number'])
    except Exception as e:
        print(f"❌ Error getting episodes: {e}")
        return []

def get_video_servers(token):
    """Get available video servers"""
    try:
        enc = enc_dec_request('enc-kai', token)
        if not enc:
            return []

        url = f"{BASE_URL}/ajax/links/list?token={token}&_={enc}"
        response = scraper.get(url, headers=headers)
        data = response.json()

        if 'result' not in data:
            return []

        html = data['result']
        soup = BeautifulSoup(html, 'html.parser')

        servers = []
        for type_div in soup.select('div.server-items[data-id]'):
            type_id = type_div.get('data-id', '')

            for server in type_div.select('span.server[data-lid]'):
                server_id = server.get('data-lid', '')
                server_name = server.text.strip()

                servers.append({
                    'type': type_id,
                    'server_id': server_id,
                    'server_name': server_name
                })

        return servers
    except Exception as e:
        print(f"❌ Error getting servers: {e}")
        return []

def get_video_url(server_id, server_name):
    """Get direct video URL"""
    try:
        enc = enc_dec_request('enc-kai', server_id)
        if not enc:
            return None

        url = f"{BASE_URL}/ajax/links/view?id={server_id}&_={enc}"
        response = scraper.get(url, headers=headers)
        data = response.json()

        encoded_link = data.get('result', '')
        if not encoded_link:
            return None

        dec_body = json.dumps({"text": encoded_link})
        dec_response = scraper.post(
            "https://enc-dec.app/api/dec-kai",
            data=dec_body,
            headers={'Content-Type': 'application/json'}
        )
        dec_data = dec_response.json()
        iframe_url = dec_data.get('result', {}).get('url', '')

        if not iframe_url:
            return None

        return extract_megaup_url(iframe_url)
    except Exception as e:
        print(f"⚠️ Error getting video URL: {e}")
        return None

def extract_megaup_url(iframe_url):
    """Extract video URL from MegaUp"""
    try:
        parsed = urlparse(iframe_url)
        token = parsed.path.split('/')[-1]

        media_url = f"{parsed.scheme}://{parsed.netloc}/media/{token}"
        response = scraper.get(media_url, headers=headers)
        data = response.json()
        mega_token = data.get('result', '')

        if not mega_token:
            return None

        dec_body = json.dumps({"text": mega_token, "agent": headers['User-Agent']})
        dec_response = scraper.post(
            "https://enc-dec.app/api/dec-mega",
            data=dec_body,
            headers={'Content-Type': 'application/json'}
        )

        mega_data = dec_response.json()
        sources = mega_data.get('result', {}).get('sources', [])

        if not sources:
            return None

        return sources[0].get('file', '')
    except Exception as e:
        print(f"⚠️ MegaUp extraction error: {e}")
        return None

print("✅ Core functions loaded")

# @title 📥 **Download Methods** { display-mode: "form" }

def download_with_ytdlp(url, output_file, episode_num):
    """Download using yt-dlp (BEST for m3u8)"""
    try:
        print(f"\n📥 Downloading Episode {episode_num} with yt-dlp...")
        print(f"   File: {os.path.basename(output_file)}")

        cmd = [
            'yt-dlp', url, '-o', output_file,
            '--no-warnings', '--no-check-certificate',
            '--concurrent-fragments', str(max_workers),
            '--retries', str(max_retries),
            '--fragment-retries', str(max_retries),
            '--socket-timeout', str(timeout),
            '--progress', '--newline',
            '--user-agent', headers['User-Agent'],
            '--referer', BASE_URL
        ]

        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

        for line in process.stdout:
            if '[download]' in line and '%' in line:
                match = re.search(r'(\d+\.\d+)%', line)
                if match:
                    percent = float(match.group(1))
                    print(f"\r   ⏳ Progress: {percent:.1f}%", end='', flush=True)

        process.wait()

        if process.returncode == 0 and os.path.exists(output_file):
            file_size = os.path.getsize(output_file) / (1024*1024)
            print(f"\n   ✅ Complete! ({file_size:.2f} MB)")
            return True
        else:
            print(f"\n   ❌ Download failed")
            return False
    except Exception as e:
        print(f"\n   ❌ Error: {e}")
        return False

def download_with_aria2(url, output_file, episode_num):
    """Download using aria2c (FAST multi-connection)"""
    try:
        print(f"\n📥 Downloading Episode {episode_num} with aria2...")
        print(f"   File: {os.path.basename(output_file)}")

        cmd = [
            'aria2c', url,
            '-o', os.path.basename(output_file),
            '-d', os.path.dirname(output_file),
            '-x', str(max_workers),
            '-s', str(max_workers),
            '--max-tries=5', '--retry-wait=3',
            '--user-agent=' + headers['User-Agent'],
            '--referer=' + BASE_URL
        ]

        subprocess.run(cmd, check=False)

        if os.path.exists(output_file):
            file_size = os.path.getsize(output_file) / (1024*1024)
            print(f"\n   ✅ Complete! ({file_size:.2f} MB)")
            return True
        return False
    except Exception as e:
        print(f"\n   ❌ Error: {e}")
        return False

def download_chunk(url, start, end, chunk_file, pbar):
    """Download a single chunk"""
    try:
        chunk_headers = headers.copy()
        chunk_headers['Range'] = f'bytes={start}-{end}'
        response = scraper.get(url, headers=chunk_headers, stream=True, timeout=30)

        if response.status_code not in [200, 206]:
            return False

        with open(chunk_file, 'wb') as f:
            for data in response.iter_content(chunk_size=8192):
                if data:
                    f.write(data)
                    pbar.update(len(data))
        return True
    except:
        return False

def download_with_chunks(url, output_file, episode_num):
    """Download with chunked/parallel downloading"""
    try:
        print(f"\n📥 Downloading Episode {episode_num} with chunks...")
        print(f"   File: {os.path.basename(output_file)}")

        head_response = scraper.head(url, headers=headers, timeout=10)

        if 'Content-Length' not in head_response.headers:
            return download_direct(url, output_file, episode_num)

        total_size = int(head_response.headers['Content-Length'])
        chunk_size = chunk_size_mb * 1024 * 1024

        chunks = []
        for i in range(0, total_size, chunk_size):
            start = i
            end = min(i + chunk_size - 1, total_size - 1)
            chunks.append((start, end))

        chunk_dir = f"{output_file}.chunks"
        os.makedirs(chunk_dir, exist_ok=True)
        chunk_files = []

        with tqdm(total=total_size, unit='B', unit_scale=True, desc="   ⏳ Downloading", ncols=80) as pbar:
            with ThreadPoolExecutor(max_workers=max_workers) as executor:
                futures = {}
                for idx, (start, end) in enumerate(chunks):
                    chunk_file = f"{chunk_dir}/chunk_{idx:04d}"
                    chunk_files.append(chunk_file)
                    future = executor.submit(download_chunk, url, start, end, chunk_file, pbar)
                    futures[future] = idx

                for future in as_completed(futures):
                    if not future.result():
                        shutil.rmtree(chunk_dir, ignore_errors=True)
                        return False

        print("   🔗 Merging chunks...")
        with open(output_file, 'wb') as outfile:
            for chunk_file in chunk_files:
                with open(chunk_file, 'rb') as infile:
                    shutil.copyfileobj(infile, outfile)

        shutil.rmtree(chunk_dir, ignore_errors=True)

        file_size = os.path.getsize(output_file) / (1024*1024)
        print(f"   ✅ Complete! ({file_size:.2f} MB)")
        return True
    except Exception as e:
        print(f"\n   ❌ Error: {e}")
        return False

def download_direct(url, output_file, episode_num):
    """Direct download with progress"""
    try:
        print(f"\n📥 Downloading Episode {episode_num}...")
        response = scraper.get(url, headers=headers, stream=True, timeout=30)
        total_size = int(response.headers.get('content-length', 0))

        with open(output_file, 'wb') as f:
            with tqdm(total=total_size, unit='B', unit_scale=True, desc="   ⏳", ncols=80) as pbar:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
                        pbar.update(len(chunk))

        file_size = os.path.getsize(output_file) / (1024*1024)
        print(f"   ✅ Complete! ({file_size:.2f} MB)")
        return True
    except Exception as e:
        print(f"\n   ❌ Error: {e}")
        return False

def download_episode(url, output_file, episode_num):
    """Main download with retry"""
    is_m3u8 = '.m3u8' in url

    if download_method == "yt-dlp":
        download_func = download_with_ytdlp
    elif download_method == "aria2" and not is_m3u8:
        download_func = download_with_aria2
    elif download_method == "chunks" and not is_m3u8:
        download_func = download_with_chunks
    else:
        download_func = download_with_ytdlp if is_m3u8 else download_with_chunks

    for attempt in range(1, max_retries + 1):
        if attempt > 1:
            print(f"\n   🔄 Retry {attempt}/{max_retries}...")
            time.sleep(3)

        if os.path.exists(output_file):
            os.remove(output_file)

        if download_func(url, output_file, episode_num):
            return True

    return False

print("✅ Download methods loaded")

# @title 📺 **Fetch Anime Info & Download** { display-mode: "form" }

print("\n" + "=" * 70)
print("🎬 ANIMEKAI EPISODE DOWNLOADER")
print("=" * 70)

print(f"\n🔍 Processing: {anime_url}")

anime_id, anime_title = get_anime_details(anime_url)

if not anime_id:
    raise Exception("❌ Could not extract anime ID")

print(f"✅ Anime ID: {anime_id}")
print(f"📺 Title: {anime_title}")

episode_list = get_episode_list(anime_id)

if not episode_list:
    raise Exception("❌ No episodes found!")

print(f"📋 Found {len(episode_list)} episode(s)")

# Determine episodes
if download_mode == "Single Episode":
    episodes_to_download = [ep for ep in episode_list if ep['number'] == single_episode]
elif download_mode == "Episode Range":
    episodes_to_download = [ep for ep in episode_list if start_episode <= ep['number'] <= end_episode]
else:
    episodes_to_download = episode_list

if not episodes_to_download:
    raise Exception("❌ No episodes match selection!")

print(f"📥 Will download {len(episodes_to_download)} episode(s)")

download_dir = f"downloads/{anime_title}"
os.makedirs(download_dir, exist_ok=True)

type_map = {"Hard Sub": "sub", "Soft Sub": "softsub", "Dub & S-Sub": "dub"}
prefer_type_id = type_map.get(prefer_type, "softsub")

downloaded_files = []
failed_episodes = []

for idx, episode in enumerate(episodes_to_download, 1):
    print(f"\n[{idx}/{len(episodes_to_download)}] Episode {episode['number']}")

    try:
        servers = get_video_servers(episode['token'])
        if not servers:
            failed_episodes.append(episode['number'])
            continue

        # Map preference types
        # Note: "dub" type on AnimeKai means Dub & S-Sub (dual audio with soft subs)
        type_map_search = {
            "Hard Sub": "sub",
            "Soft Sub": "softsub",
            "Dub & S-Sub": "dub"  # This is the dual audio option
        }
        prefer_type_id = type_map_search.get(prefer_type, "softsub")

        # Filter servers by preference
        matching_servers = [s for s in servers if s['type'] == prefer_type_id and s['server_name'] == prefer_server]

        # Fallback 1: Try any server with matching type
        if not matching_servers:
            matching_servers = [s for s in servers if s['type'] == prefer_type_id]

        # Fallback 2: Try any server with preferred server name
        if not matching_servers:
            matching_servers = [s for s in servers if s['server_name'] == prefer_server]

        # Fallback 3: Use first available server
        if not matching_servers:
            matching_servers = servers[:1]

        server = matching_servers[0]

        # Show what type we're actually downloading
        type_display = {
            "sub": "Hard Sub",
            "softsub": "Soft Sub",
            "dub": "Dub & S-Sub (Dual Audio)"
        }.get(server['type'], server['type'])

        print(f"   🎥 Server: {server['server_name']} | Type: {type_display}")

        video_url = get_video_url(server['server_id'], server['server_name'])
        if not video_url:
            failed_episodes.append(episode['number'])
            continue

        ep_num_str = f"{int(episode['number']):03d}"

        # Add type to filename for clarity
        type_suffix = {
            "sub": "HardSub",
            "softsub": "SoftSub",
            "dub": "DualAudio"
        }.get(server['type'], "")

        filename = f"{download_dir}/Episode_{ep_num_str}_{type_suffix}.mp4"

        if download_episode(video_url, filename, episode['number']):
            downloaded_files.append(filename)
        else:
            failed_episodes.append(episode['number'])

        time.sleep(2)
    except Exception as e:
        print(f"   ❌ Error: {e}")
        failed_episodes.append(episode['number'])

print("\n" + "=" * 70)
print("📊 DOWNLOAD SUMMARY")
print("=" * 70)
print(f"\n✅ Downloaded: {len(downloaded_files)} episode(s)")
if failed_episodes:
    print(f"❌ Failed: {', '.join(map(str, failed_episodes))}")
if downloaded_files:
    total_size = sum(os.path.getsize(f) for f in downloaded_files) / (1024*1024)
    print(f"💾 Total: {total_size:.2f} MB")
print("\n🎉 COMPLETE!")

📦 Installing required packages...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.8 MB/s eta 0:00:00
✅ All dependencies installed!

✅ Configuration loaded!
✅ Core functions loaded
✅ Download methods loaded

🎬 ANIMEKAI EPISODE DOWNLOADER

🔍 Processing: https://anikai.to/watch/attack-on-titan-season-3-oxgk
✅ Anime ID: eoI
📺 Title: Unknown
📋 Found 12 episode(s)
📥 Will download 12 episode(s)

[1/12] Episode 1.0
   🎥 Server: Server 1 | Type: Dub & S-Sub (Dual Audio)

📥 Downloading Episode 1.0 with yt-dlp...
   File: Episode_001_DualAudio.mp4
   ⏳ Progress: 99.5%
   ✅ Complete! (262.46 MB)

[2/12] Episode 2.0
   🎥 Server: Server 1 | Type: Dub & S-Sub (Dual Audio)

📥 Downloading Episode 2.0 with yt-dlp...
   File: Episode_002_DualAudio.mp4
   ⏳ Progre

In [ ]:

# Manhwa Scraper, Merger & Uploader (Batch Mode) - ADVANCED PRIORITY MODE
# Scrapes manhwa chapters from Comick.art, merges into PDF/CBZ in batches, uploads to GDrive/Gofile

#@title **Install Dependencies** { display-mode: "form" }
!pip install -q pillow beautifulsoup4 requests pyrogram tgcrypto langdetect img2pdf PyPDF2
!apt-get install -y fonts-noto-cjk

#@title **Import Libraries** { display-mode: "form" }
import os
import re
import json
import shutil
import zipfile
import requests
import traceback
from pathlib import Path
from PIL import Image, ImageDraw
from bs4 import BeautifulSoup
from google.colab import drive, files
import img2pdf
from PyPDF2 import PdfMerger
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock
from collections import defaultdict
from datetime import datetime

#@title **Configuration** { display-mode: "form" }
#@markdown Enter the Comick.art manga URL
manga_url = "https://comick.art/comic/03-omniscient-reader-s-viewpoint" #@param {type:"string"}

#@markdown Scanlation Group Filter (leave empty to use priority mode)
preferred_group = "" #@param {type:"string"}

#@markdown Priority Mode (if group filter is empty)
priority_mode = "Mix Best Available" #@param ["Most Upvoted", "Latest Upload", "Mix Best Available"]

#@markdown Exclude groups (comma-separated, e.g., "Official,WEBTOON,LINE Webtoon,Colored Manga")
exclude_groups = "Official,WEBTOON,LINE Webtoon" #@param {type:"string"}

#@markdown Only include numbered chapters (exclude side stories, prologues, etc.)
only_numbered_chapters = False #@param {type:"boolean"}

#@markdown Chapters per file (creates separate files for every N chapters)
chapters_per_file = 100 #@param {type:"integer"}

#@markdown Starting chapter (1 = from beginning)
start_chapter = 1 #@param {type:"integer"}

#@markdown Total chapters to download (0 = all available)
total_chapters = 0 #@param {type:"integer"}

#@markdown Upload destination
upload_to = "Gofile" #@param ["GDrive", "Gofile", "Both", "None"]

#@markdown Create PDF
create_pdf = False #@param {type:"boolean"}

#@markdown Create CBZ
create_cbz = True #@param {type:"boolean"}

#@markdown Number of parallel downloads (higher = faster, but may cause issues)
max_workers = 10 #@param {type:"integer"}

#@title **Core Functions** { display-mode: "form" }

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
    'Referer': 'https://comick.art/',
}

def fetch_content(url, headers=None):
    """Fetch content from URL"""
    try:
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        return response.content
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

def get_manga_info(url):
    """Extract manga information"""
    try:
        content = fetch_content(url, headers=headers)
        soup = BeautifulSoup(content, "html.parser")
        script_tag = soup.find("script", {"id": "comic-data"})

        if not script_tag:
            return None

        comic = json.loads(script_tag.string)
        slug = comic['slug']
        title = comic['title'].strip().split("[")[0] if "[" in comic['title'] else comic['title']

        comic_url = f'https://comick.art/api/comics/{slug}/chapter-list?lang=en'

        return {
            'title': title,
            'slug': slug,
            'comic_url': comic_url
        }
    except Exception as e:
        print(f"Error getting manga info: {e}")
        return None

def is_numeric_chapter(ch_num):
    """Check if chapter number is numeric (not prologue/special)"""
    try:
        float(ch_num)
        return True
    except (ValueError, TypeError):
        return False

def parse_date(date_str):
    """Parse date string to datetime object"""
    try:
        return datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%fZ")
    except:
        try:
            return datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")
        except:
            return datetime.min

def get_chapters(comic_url, preferred_group=None, exclude_groups_str="", priority_mode_str="Mix Best Available", only_numbered=False):
    """Get chapter URLs with advanced priority modes"""
    try:
        content = fetch_content(comic_url, headers=headers)
        first_json = json.loads(content)

        last_page = int(first_json['pagination']['last_page'])
        manga_hid = comic_url.split("/")[-2]

        # Parse exclude groups
        excluded = set()
        if exclude_groups_str:
            excluded = set(g.strip().lower() for g in exclude_groups_str.split(',') if g.strip())

        # Store all chapters per chapter number
        chapters_dict = defaultdict(list)

        print(f"  📄 Scanning {last_page} pages...")

        for page in range(1, last_page + 1):
            if page % 5 == 0:
                print(f"  ⏳ Progress: {page}/{last_page} pages...", end='\r')

            page_url = f"{comic_url}&page={page}"
            content = fetch_content(page_url, headers=headers)
            data = json.loads(content).get("data", [])

            for chapter in data:
                # Only English
                if chapter.get("lang") != "en":
                    continue

                # Get chapter number
                ch_num = chapter.get("chap")
                if ch_num is None or ch_num == "":
                    ch_num = chapter.get("title", "0")

                ch_num_str = str(ch_num)

                # Skip non-numeric if only_numbered is True
                if only_numbered and not is_numeric_chapter(ch_num):
                    continue

                groups = chapter.get("group_name", [])

                # Skip excluded groups
                if excluded and any(g.lower() in excluded for g in groups):
                    continue

                # If preferred group specified, skip if not from that group
                if preferred_group and preferred_group not in groups:
                    continue

                # Store chapter with metadata
                up_count = chapter.get("up_count", 0)
                down_count = chapter.get("down_count", 0)
                group_str = ', '.join(groups) if groups else 'Unknown'
                created_at = chapter.get("created_at", "")

                chapters_dict[ch_num_str].append({
                    'hid': chapter["hid"],
                    'chap': chapter.get("chap", "0"),
                    'ch_num': ch_num,
                    'group': group_str,
                    'up_count': up_count,
                    'down_count': down_count,
                    'created_at': created_at,
                    'lang': chapter.get("lang", "en"),
                    'groups_raw': groups
                })

        print(f"\n  ✅ Found {len(chapters_dict)} unique chapter numbers")

        # Preferred groups for Mix Best Available (in priority order)
        preferred_groups_priority = [
            'flame comics',
            'asura',
            'reaper scans',
            'luminous scans',
            'void scans'
        ]

        # Now select best chapter for each number
        urls = []
        selection_stats = defaultdict(int)

        for ch_num_str, chapter_list in chapters_dict.items():
            selected_chapter = None

            if preferred_group:
                # Use specific group
                selected_chapter = chapter_list[0]
            elif priority_mode_str == "Most Upvoted":
                # Highest upvotes
                selected_chapter = max(chapter_list, key=lambda x: (x['up_count'], -x['down_count']))
            elif priority_mode_str == "Latest Upload":
                # Most recent upload
                selected_chapter = max(chapter_list, key=lambda x: parse_date(x['created_at']))
            else:  # Mix Best Available
                # Smart selection: prefer quality groups, then upvotes

                # First, try to find from preferred groups
                for pref_group in preferred_groups_priority:
                    candidates = [ch for ch in chapter_list
                                if any(pref_group in g.lower() for g in ch['groups_raw'])]
                    if candidates:
                        # Among preferred group, choose highest upvoted
                        selected_chapter = max(candidates, key=lambda x: x['up_count'])
                        break

                # If no preferred group found, fallback to highest upvoted
                if not selected_chapter:
                    selected_chapter = max(chapter_list, key=lambda x: (x['up_count'], -x['down_count']))

            if selected_chapter:
                link = f'https://comick.art/comic/{manga_hid}/{selected_chapter["hid"]}-chapter-{selected_chapter["chap"]}-{selected_chapter["lang"]}'
                urls.append((link, selected_chapter['ch_num'], selected_chapter['group'], selected_chapter['up_count']))
                selection_stats[selected_chapter['group']] += 1

        # Sort by chapter number
        def sort_key(item):
            try:
                return (0, float(item[1]))  # Numeric chapters
            except (ValueError, TypeError):
                return (1, str(item[1]))  # Non-numeric chapters

        urls.sort(key=sort_key)

        print(f"\n  📊 Selected chapters by group:")
        for group, count in sorted(selection_stats.items(), key=lambda x: x[1], reverse=True)[:10]:
            print(f"    • {group}: {count} chapters")

        return urls
    except Exception as e:
        print(f"Error getting chapters: {e}")
        traceback.print_exc()
        return []

def get_chapter_images(chapter_url):
    """Get image URLs from chapter"""
    try:
        content = fetch_content(chapter_url, headers=headers)
        soup = BeautifulSoup(content, "html.parser")
        script_tag = soup.find("script", {"id": "sv-data"})

        if not script_tag:
            return []

        data = json.loads(script_tag.string)

        if "chap" in data:
            return []

        images = [img["url"] for img in data["chapter"]["images"] if img.get("url")]
        return images
    except Exception as e:
        print(f"Error getting chapter images: {e}")
        return []

def download_image(url, filepath, headers=None, retry_count=3):
    """Download single image with retries"""
    for attempt in range(retry_count):
        try:
            download_headers = headers.copy() if headers else {}
            download_headers.update({
                'Accept': 'image/avif,image/webp,image/apng,image/svg+xml,image/*,*/*;q=0.8',
                'Accept-Language': 'en-US,en;q=0.9',
                'Connection': 'keep-alive',
            })

            response = requests.get(url, headers=download_headers, timeout=30, stream=True)

            if response.status_code == 200:
                with open(filepath, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)

                if os.path.exists(filepath) and os.path.getsize(filepath) > 0:
                    return True

            if attempt < retry_count - 1:
                continue

        except Exception as e:
            if attempt < retry_count - 1:
                continue

    return False

def download_image_wrapper(args):
    """Wrapper for parallel downloading"""
    url, filepath, headers = args
    return download_image(url, filepath, headers)

def download_chapter_parallel(images, batch_dir, ch_formatted, headers, max_workers=10):
    """Download all images of a chapter in parallel"""
    download_tasks = []

    for img_idx, img_url in enumerate(images, 1):
        img_path = os.path.join(batch_dir, f"ch{ch_formatted}_p{img_idx:03d}.jpg")
        download_tasks.append((img_url, img_path, headers))

    downloaded = 0
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(download_image_wrapper, task): task for task in download_tasks}

        for future in as_completed(futures):
            if future.result():
                downloaded += 1

    return downloaded

def create_pdf_from_images(image_folder, output_path, title):
    """Create PDF from images with progress tracking and automatic image fixing"""
    try:
        image_files = sorted([f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png', '.jpeg', '.webp'))])

        if not image_files:
            print("  ⚠️ No images found")
            return False

        total_images = len(image_files)
        print(f"  📊 Processing {total_images} images...")

        temp_folder = os.path.join(image_folder, 'temp_pdf')
        os.makedirs(temp_folder, exist_ok=True)

        valid_image_paths = []
        fixed_count = 0

        for idx, img_file in enumerate(image_files, 1):
            img_path = os.path.join(image_folder, img_file)
            temp_path = os.path.join(temp_folder, f"fixed_{idx:05d}.jpg")

            try:
                with Image.open(img_path) as img:
                    width, height = img.size

                    min_dimension = 100
                    needs_resize = False

                    if width < min_dimension or height < min_dimension:
                        scale = max(min_dimension / width, min_dimension / height)
                        new_width = max(int(width * scale), min_dimension)
                        new_height = max(int(height * scale), min_dimension)

                        img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
                        needs_resize = True
                        fixed_count += 1

                    if img.mode != 'RGB':
                        img = img.convert('RGB')
                        needs_resize = True

                    if needs_resize or img.mode != 'RGB':
                        img.save(temp_path, 'JPEG', quality=95, optimize=True)
                        valid_image_paths.append(temp_path)
                    else:
                        valid_image_paths.append(img_path)

                if idx % 100 == 0 or idx == total_images:
                    print(f"  ⏳ Processed {idx}/{total_images} images...", end='\r')

            except Exception as e:
                print(f"\n  ⚠️ Corrupted image {img_file}, creating placeholder...")

                placeholder = Image.new('RGB', (800, 1200), color=(240, 240, 240))
                draw = ImageDraw.Draw(placeholder)

                text = f"Image {idx}\nCould not load\n{img_file[:30]}"
                draw.text((400, 600), text, fill=(128, 128, 128), anchor="mm")

                placeholder.save(temp_path, 'JPEG', quality=95)
                valid_image_paths.append(temp_path)
                fixed_count += 1

        if not valid_image_paths:
            print("\n  ❌ No valid images to create PDF")
            shutil.rmtree(temp_folder, ignore_errors=True)
            return False

        if fixed_count > 0:
            print(f"\n  🔧 Fixed/processed {fixed_count} images for PDF compatibility")

        print(f"\n  🔄 Converting {len(valid_image_paths)} images to PDF in batches...")

        batch_size = 100
        temp_pdfs = []
        pdf_temp_folder = os.path.join(temp_folder, 'pdf_batches')
        os.makedirs(pdf_temp_folder, exist_ok=True)

        for i in range(0, len(valid_image_paths), batch_size):
            batch_paths = valid_image_paths[i:i + batch_size]
            progress = min(i + batch_size, len(valid_image_paths))
            print(f"  📄 Processing batch {progress}/{len(valid_image_paths)} images...", end='\r')

            batch_pdf_path = os.path.join(pdf_temp_folder, f"batch_{i:05d}.pdf")
            with open(batch_pdf_path, "wb") as f:
                f.write(img2pdf.convert(batch_paths))
            temp_pdfs.append(batch_pdf_path)

        print(f"\n  🔗 Merging {len(temp_pdfs)} PDF batches into final file...")

        merger = PdfMerger()
        for idx, pdf_path in enumerate(temp_pdfs, 1):
            print(f"  📚 Merging batch {idx}/{len(temp_pdfs)}...", end='\r')
            merger.append(pdf_path)

        print(f"\n  💾 Writing final PDF...")
        merger.write(output_path)
        merger.close()

        shutil.rmtree(temp_folder, ignore_errors=True)

        file_size = os.path.getsize(output_path) / (1024 * 1024)
        print(f"  ✅ PDF created: {os.path.basename(output_path)} ({file_size:.2f} MB)")
        print(f"  📄 Total pages: {len(valid_image_paths)}")
        return True
    except Exception as e:
        print(f"  ❌ Error creating PDF: {e}")
        print(f"  🔍 Error details: {traceback.format_exc()}")
        try:
            temp_folder = os.path.join(image_folder, 'temp_pdf')
            shutil.rmtree(temp_folder, ignore_errors=True)
        except:
            pass
        return False

def create_cbz_from_images(image_folder, output_path, title):
    """Create CBZ from images with progress tracking"""
    try:
        image_files = []
        for root, dirs, files in os.walk(image_folder):
            for file in sorted(files):
                if file.endswith(('.jpg', '.png', '.jpeg', '.webp')):
                    image_files.append(os.path.join(root, file))

        if not image_files:
            print("  ⚠️ No images found")
            return False

        total_images = len(image_files)
        print(f"  📊 Compressing {total_images} images...")

        with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED, compresslevel=6) as zipf:
            for idx, file_path in enumerate(image_files, 1):
                arcname = os.path.relpath(file_path, image_folder)
                zipf.write(file_path, arcname)

                if idx % 50 == 0 or idx == total_images:
                    print(f"  ⏳ Compressed {idx}/{total_images} images...", end='\r')

        file_size = os.path.getsize(output_path) / (1024 * 1024)
        print(f"\n  ✅ CBZ created: {os.path.basename(output_path)} ({file_size:.2f} MB)")
        return True
    except Exception as e:
        print(f"  ❌ Error creating CBZ: {e}")
        return False

def upload_to_gofile(filepath, retry_count=3):
    """Upload file to Gofile.io with retries"""
    for attempt in range(retry_count):
        try:
            server_response = requests.get("https://api.gofile.io/servers", timeout=30)
            server_data = server_response.json()

            if server_data['status'] != 'ok':
                print(f"  ⚠️ Attempt {attempt + 1}/{retry_count}: Failed to get Gofile server")
                if attempt < retry_count - 1:
                    print(f"  ⏳ Retrying in 5 seconds...")
                    import time
                    time.sleep(5)
                    continue
                return None

            servers = server_data['data']['servers']
            if not servers:
                print(f"  ⚠️ Attempt {attempt + 1}/{retry_count}: No servers available")
                if attempt < retry_count - 1:
                    print(f"  ⏳ Retrying in 5 seconds...")
                    import time
                    time.sleep(5)
                    continue
                return None

            server = servers[0]['name']
            upload_url = f"https://{server}.gofile.io/contents/uploadfile"

            print(f"  📤 Uploading {os.path.basename(filepath)} (Attempt {attempt + 1}/{retry_count})...")

            with open(filepath, 'rb') as f:
                files_data = {'file': f}
                response = requests.post(upload_url, files=files_data, timeout=600)

            result = response.json()

            if result['status'] == 'ok':
                download_page = result['data']['downloadPage']
                print(f"  ✅ Gofile: {download_page}")
                return download_page
            else:
                print(f"  ⚠️ Attempt {attempt + 1}/{retry_count}: Upload failed - {result.get('status', 'unknown error')}")
                if attempt < retry_count - 1:
                    print(f"  ⏳ Retrying in 5 seconds...")
                    import time
                    time.sleep(5)
                    continue
                return None

        except requests.exceptions.Timeout:
            print(f"  ⚠️ Attempt {attempt + 1}/{retry_count}: Upload timeout")
            if attempt < retry_count - 1:
                print(f"  ⏳ Retrying in 10 seconds...")
                import time
                time.sleep(10)
                continue
            return None
        except Exception as e:
            print(f"  ⚠️ Attempt {attempt + 1}/{retry_count}: Error - {e}")
            if attempt < retry_count - 1:
                print(f"  ⏳ Retrying in 5 seconds...")
                import time
                time.sleep(5)
                continue
            return None

    print(f"  ❌ Failed to upload after {retry_count} attempts")
    return None

#@title **Main Scraping Process** { display-mode: "form" }

print("=" * 70)
print("🎯 MANHWA BATCH SCRAPER & UPLOADER")
print("=" * 70)

print(f"\n📖 Fetching manga info from: {manga_url}")
manga_info = get_manga_info(manga_url)

if not manga_info:
    print("❌ Failed to get manga info")
else:
    title = manga_info['title']
    print(f"✅ Title: {title}")

    if preferred_group:
        print(f"\n🎯 Using specific group: {preferred_group}")
    else:
        print(f"\n🎯 Priority mode: {priority_mode}")

    if exclude_groups:
        print(f"🚫 Excluding groups: {exclude_groups}")

    if only_numbered_chapters:
        print(f"🔢 Filter: Only numbered chapters (skipping prologue/side stories)")

    print(f"🌐 Language: English only")

    print(f"\n📚 Fetching and analyzing chapters...")
    all_chapters = get_chapters(manga_info['comic_url'], preferred_group, exclude_groups, priority_mode, only_numbered_chapters)
    print(f"✅ Total unique chapters selected: {len(all_chapters)}")

    # Apply chapter range
    if start_chapter > 1 or total_chapters > 0:
        # Find chapters in range
        filtered_chapters = []
        for url, ch_num, group, upvotes in all_chapters:
            try:
                ch_float = float(ch_num)
                if ch_float >= start_chapter:
                    if total_chapters == 0 or ch_float < start_chapter + total_chapters:
                        filtered_chapters.append((url, ch_num, group, upvotes))
            except:
                # Non-numeric chapters - include if no filtering
                if start_chapter <= 1:
                    filtered_chapters.append((url, ch_num, group, upvotes))

        selected_chapters = filtered_chapters
    else:
        selected_chapters = all_chapters

    print(f"\n📥 Will download {len(selected_chapters)} chapters (starting from chapter {start_chapter})")
    print(f"📦 Creating files with {chapters_per_file} chapters each")

    # Show sample
    print(f"\n📋 Sample chapters (first 5):")
    for i, (url, ch_num, group, upvotes) in enumerate(selected_chapters[:5], 1):
        print(f"  {i}. Chapter {ch_num} - [{group}] (👍 {upvotes})")

    if len(selected_chapters) > 5:
        print(f"  ...")
        for i, (url, ch_num, group, upvotes) in enumerate(selected_chapters[-2:], len(selected_chapters)-1):
            print(f"  {i}. Chapter {ch_num} - [{group}] (👍 {upvotes})")

    base_download_dir = f"/content/{title}"
    output_dir = "/content/output"
    os.makedirs(output_dir, exist_ok=True)

    batches = []
    for i in range(0, len(selected_chapters), chapters_per_file):
        batch = selected_chapters[i:i + chapters_per_file]
        batches.append(batch)

    print(f"\n📊 Total batches: {len(batches)}")

    gdrive_dir = None
    if upload_to in ["GDrive", "Both"]:
        print("\n☁️ Mounting Google Drive...")
        drive.mount('/content/gdrive')
        gdrive_dir = "/content/gdrive/MyDrive/Manhwa"
        os.makedirs(gdrive_dir, exist_ok=True)
        print(f"✅ Google Drive mounted at: {gdrive_dir}")

    safe_title = re.sub(r'[<>:"/\\|?*]', '', title)

    # Store all upload links
    all_upload_links = {}

    for batch_num, batch in enumerate(batches, 1):
        print("\n" + "=" * 70)
        print(f"📦 BATCH {batch_num}/{len(batches)}")

        first_ch = batch[0][1]
        last_ch = batch[-1][1]

        try:
            first_ch_display = f"{float(first_ch):.1f}".rstrip('0').rstrip('.')
        except:
            first_ch_display = str(first_ch)

        try:
            last_ch_display = f"{float(last_ch):.1f}".rstrip('0').rstrip('.')
        except:
            last_ch_display = str(last_ch)

        batch_name = f"{safe_title} Chapters {first_ch_display}-{last_ch_display}"
        print(f"📖 Chapters: {first_ch_display} to {last_ch_display} ({len(batch)} chapters)")
        print("=" * 70)

        batch_dir = os.path.join(base_download_dir, f"batch_{batch_num}")
        os.makedirs(batch_dir, exist_ok=True)

        total_images = 0
        for idx, (ch_url, ch_num, group, upvotes) in enumerate(batch, 1):
            print(f"\n[{idx}/{len(batch)}] Chapter {ch_num} [{group}] (👍 {upvotes})")

            images = get_chapter_images(ch_url)
            if not images:
                print(f"  ⚠️ No images found")
                continue

            print(f"  📷 Found {len(images)} images - downloading in parallel...")

            try:
                ch_formatted = f"{float(ch_num):06.1f}"
            except (ValueError, TypeError):
                ch_formatted = re.sub(r'[<>:"/\\|?*]', '', str(ch_num)).zfill(6)

            downloaded = download_chapter_parallel(images, batch_dir, ch_formatted, headers, max_workers)
            total_images += downloaded

            print(f"  ✅ Downloaded {downloaded}/{len(images)} images")

        print(f"\n✅ Batch total: {total_images} images")

        uploaded_files = []
        batch_links = []

        if create_pdf:
            print(f"\n📄 Creating PDF for batch {batch_num}...")
            pdf_path = os.path.join(output_dir, f"{batch_name}.pdf")
            if create_pdf_from_images(batch_dir, pdf_path, batch_name):
                uploaded_files.append(('PDF', pdf_path))

        if create_cbz:
            print(f"\n📦 Creating CBZ for batch {batch_num}...")
            cbz_path = os.path.join(output_dir, f"{batch_name}.cbz")
            if create_cbz_from_images(batch_dir, cbz_path, batch_name):
                uploaded_files.append(('CBZ', cbz_path))

        if upload_to in ["GDrive", "Both"] and gdrive_dir:
            print(f"\n☁️ Uploading batch {batch_num} to Google Drive...")
            for file_type, filepath in uploaded_files:
                dest = os.path.join(gdrive_dir, os.path.basename(filepath))
                shutil.copy(filepath, dest)
                gdrive_link = dest
                batch_links.append((f"{file_type} (GDrive)", gdrive_link))
                print(f"  ✅ {file_type}: {dest}")

        if upload_to in ["Gofile", "Both"]:
            print(f"\n🌐 Uploading batch {batch_num} to Gofile.io...")
            for file_type, filepath in uploaded_files:
                gofile_link = upload_to_gofile(filepath)
                if gofile_link:
                    batch_links.append((f"{file_type} (Gofile)", gofile_link))

        if batch_links:
            all_upload_links[batch_num] = batch_links

        if upload_to == "None":
            print(f"\n💾 Batch {batch_num} files saved locally in /content/output/")

        print(f"\n🧹 Cleaning up batch {batch_num} temporary files...")
        shutil.rmtree(batch_dir, ignore_errors=True)

    print("\n" + "=" * 70)
    print("✅ ALL BATCHES COMPLETED!")
    print("=" * 70)
    print(f"\n📊 Summary:")
    print(f"  • Total batches: {len(batches)}")
    print(f"  • Chapters processed: {len(selected_chapters)}")
    print(f"  • Files created: {len(batches) * len([x for x in [create_pdf, create_cbz] if x])}")

    if all_upload_links:
        print("\n🔗 UPLOAD LINKS:")
        print("=" * 70)
        for batch_num, links in all_upload_links.items():
            print(f"\n📦 Batch {batch_num}:")
            for file_type, link in links:
                print(f"  • {file_type}: {link}")
        print("\n" + "=" * 70)

    if upload_to == "None":
        print("\n📥 Downloading all files to your computer...")
        for file in os.listdir(output_dir):
            filepath = os.path.join(output_dir, file)
            if os.path.isfile(filepath):
                files.download(filepath)

    print("\n🧹 Final cleanup...")
    shutil.rmtree(base_download_dir, ignore_errors=True)

    print("\n✨ Done! Enjoy your manhwa collection!")
    print("=" * 70)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-noto-cjk is already the newest version (1:20220127+repack1-1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.
🎯 MANHWA BATCH SCRAPER & UPLOADER

📖 Fetching manga info from: https://comick.art/comic/03-omniscient-reader-s-viewpoint
✅ Title: Omniscient Reader's Viewpoint

🎯 Priority mode: Mix Best Available
🚫 Excluding groups: Official,WEBTOON,LINE Webtoon
🌐 Language: English only

📚 Fetching and analyzing chapters...
  📄 Scanning 36 pages...
  ⏳ Progress: 35/36 pages...
  ✅ Found 394 unique chapter numbers

  📊 Selected chapters by group:
    • Flame Comics: 204 chapters
    • Mangakakalot: 99 chapters
    • Asura: 91 chapters
✅ Total unique chapters selected: 394

📥 Will download 394 chapters (starting from chapter 1)
📦 Creating files with 100 chapters each

📋 Sample chapters (first 5):
  1. Chapter 0 - [Flame Comics] (👍 0)
  2. Chapter 0.5 - [Flame Comics] (👍 0)
  3. 